### Imports

In [130]:
import pandas as pd

In [131]:
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

### Leemos y consolidamos los datasets

In [132]:
ventas = pd.read_csv('../Datasets/sell-in.txt', sep='\t')
productos = pd.read_csv('../Datasets/tb_productos.txt', sep='\t')
stock = pd.read_csv('../Datasets/tb_stocks.txt', sep='\t')
productos_predecir = pd.read_csv('../Datasets/productos_a_predecir.csv')

ventas_grouped = ventas.groupby(['periodo', 'product_id']).agg({
    'plan_precios_cuidados': 'sum',
    'cust_request_qty': 'sum', # sirve de algo para el modelo? En zulip se me menciona que es la cantidad de solicitudes que hace el cliente, por lo que creo que no
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()

ventas_producto = pd.merge(ventas_grouped, productos, on=['product_id'], how='inner')
final_dataset = pd.merge(ventas_producto, stock, on=['product_id', 'periodo'], how='left')
final_dataset = pd.merge(final_dataset, productos_predecir, on=['product_id'], how='inner')

final_dataset.to_csv('../Datasets/final_dataset.csv', sep='\t', index=False)

In [133]:
final_dataset.isnull().sum()

periodo                      0
product_id                   0
plan_precios_cuidados        0
cust_request_qty             0
cust_request_tn              0
tn                           0
cat1                         0
cat2                         0
cat3                         0
brand                        0
sku_size                     0
stock_final              17561
dtype: int64

### Analisis exploratorio basico

#### Analisis inicial

In [134]:
final_dataset.head(20)

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,20001,0,479,937.73,934.77,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
1,201702,20001,0,432,833.72,798.02,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
2,201703,20001,0,509,1330.75,1303.36,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
3,201704,20001,0,279,1132.94,1069.96,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
4,201705,20001,0,701,1550.69,1502.20,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
5,201706,20001,0,570,1575.83,1520.07,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
6,201707,20001,0,381,1086.47,1030.67,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
7,201708,20001,0,643,1289.67,1267.39,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
8,201709,20001,0,381,1356.96,1316.95,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
9,201710,20001,0,273,1441.60,1439.76,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN


In [135]:
final_dataset[final_dataset['cust_request_tn'] != final_dataset['tn']]

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,20001,0,479,937.73,934.77,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
1,201702,20001,0,432,833.72,798.02,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
2,201703,20001,0,509,1330.75,1303.36,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
3,201704,20001,0,279,1132.94,1069.96,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
4,201705,20001,0,701,1550.69,1502.20,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
31292,201910,20888,0,30,2.12,1.58,PC,CABELLO,SHAMPOO,NIVEA,750,15.33
31293,201911,20888,0,71,7.37,7.04,PC,CABELLO,SHAMPOO,NIVEA,750,7.80
31320,201911,20972,0,274,4.03,3.78,PC,DEOS,Aero,COLBERT,90,0.04
31321,201912,20972,0,212,3.00,2.98,PC,DEOS,Aero,COLBERT,90,-0.01


In [136]:
final_dataset.describe()

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,sku_size,stock_final
count,31383.00,31383.00,31383.00,31383.00,31383.00,31383.00,31383.00,13822.00
mean,201810.89,20523.48,1.08,204.20,45.41,44.45,475.92,20.15
std,81.59,344.28,11.96,122.92,116.52,112.65,880.46,56.61
min,201701.00,20001.00,0.00,1.00,0.00,0.00,1.00,-27.31
25%,201710.00,20228.00,0.00,110.00,2.41,2.40,90.00,1.19
50%,201807.00,20480.00,0.00,188.00,10.24,10.18,250.00,5.51
75%,201904.00,20797.00,0.00,284.00,31.69,31.45,475.00,18.08
max,201912.00,21294.00,289.00,756.00,2423.71,2295.20,10000.00,1562.02


In [137]:
final_dataset[(final_dataset['plan_precios_cuidados'] != 0) & (~final_dataset['plan_precios_cuidados'].isna())]

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
432,201701,20012,172,386,491.11,476.40,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
433,201702,20012,185,349,554.78,509.04,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
434,201703,20012,205,567,665.80,649.49,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
435,201704,20012,190,496,755.66,749.92,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
1076,201909,20026,143,359,210.99,209.49,HC,ROPA LAVADO,Liquido,LIMPIEX,800,27.12
...,...,...,...,...,...,...,...,...,...,...,...,...
30596,201908,20681,67,154,9.59,9.59,HC,HOGAR,Gel,MUSCULO,250,57.12
30597,201909,20681,79,199,10.67,10.67,HC,HOGAR,Gel,MUSCULO,250,46.41
30598,201910,20681,85,189,11.50,11.50,HC,HOGAR,Gel,MUSCULO,250,30.82
30599,201911,20681,66,143,7.05,7.05,HC,HOGAR,Gel,MUSCULO,250,24.27


#### Analizo si puede existir alguna venta de un producto en el mismo periodo que no sea plan_precios_cuidados y otras que si

In [138]:
ventas[(ventas['product_id'] == 20026) & (ventas['periodo'] == 201909) & (ventas['plan_precios_cuidados'] == 0)]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


In [139]:
def has_both_types(group):
    has_cuidados = (group['plan_precios_cuidados'] != 0).any()
    has_no_cuidados = (group['plan_precios_cuidados'] == 0).any()
    # display(group)
    return has_cuidados and has_no_cuidados

filtered_groups = ventas.groupby(['product_id', 'periodo']).filter(has_both_types)
filtered_groups

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


Como observo que no existe ningun producto, en ningun periodo, que fuera a su vez precio_cuidoda y que no lo fuera, procedo a modificar esa variable en dataset_final para transformarla en boolean en vez de en suma:

In [140]:
final_dataset['plan_precios_cuidados'] = final_dataset['plan_precios_cuidados'].apply(lambda x: 1 if x > 0 else 0)

In [141]:
final_dataset[(final_dataset['plan_precios_cuidados'] != 0) & (~final_dataset['plan_precios_cuidados'].isna())]

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
432,201701,20012,1,386,491.11,476.40,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
433,201702,20012,1,349,554.78,509.04,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
434,201703,20012,1,567,665.80,649.49,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
435,201704,20012,1,496,755.66,749.92,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
1076,201909,20026,1,359,210.99,209.49,HC,ROPA LAVADO,Liquido,LIMPIEX,800,27.12
...,...,...,...,...,...,...,...,...,...,...,...,...
30596,201908,20681,1,154,9.59,9.59,HC,HOGAR,Gel,MUSCULO,250,57.12
30597,201909,20681,1,199,10.67,10.67,HC,HOGAR,Gel,MUSCULO,250,46.41
30598,201910,20681,1,189,11.50,11.50,HC,HOGAR,Gel,MUSCULO,250,30.82
30599,201911,20681,1,143,7.05,7.05,HC,HOGAR,Gel,MUSCULO,250,24.27


In [142]:
# Observo que tambien los Na de plan_precios_cuidados se mapearon a 0
final_dataset[final_dataset['plan_precios_cuidados'].isna()]

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final


#### Defino la variable target

In [143]:
final_dataset.rename(columns={'tn': 'y'}, inplace=True)
final_dataset.to_csv('../Datasets/final_dataset.csv', sep='\t', index=False)
final_dataset

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,20001,0,479,937.73,934.77,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
1,201702,20001,0,432,833.72,798.02,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
2,201703,20001,0,509,1330.75,1303.36,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
3,201704,20001,0,279,1132.94,1069.96,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
4,201705,20001,0,701,1550.69,1502.20,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
31378,201911,20728,0,73,16.02,16.02,FOODS,ADEREZOS,Ketchup,NATURA,250,38.96
31379,201912,20728,0,98,14.32,14.32,FOODS,ADEREZOS,Ketchup,NATURA,250,40.77
31380,201912,20770,0,6,3.19,3.19,HC,PROFESIONAL,LV ROPA POLVO,INDUSTRIAL,25,9.53
31381,201912,20792,0,7,3.81,3.81,HC,PROFESIONAL,LV ROPA LIQUIDO,INDUSTRIAL,5000,44.78


#### Analisis nulos

In [144]:
# TODO
final_dataset.isnull().sum()

periodo                      0
product_id                   0
plan_precios_cuidados        0
cust_request_qty             0
cust_request_tn              0
y                            0
cat1                         0
cat2                         0
cat3                         0
brand                        0
sku_size                     0
stock_final              17561
dtype: int64

In [ ]:
#generar dataset Kaggle
#df.to_csv('../Datasets/df.csv', sep=',', index=False)